In [2]:
import numpy as np 
import pandas as pd
import os

import re
from nltk.corpus import stopwords

from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

In [3]:
df = pd.read_csv('./mt_samples_preproc.csv',encoding='iso-8859-1')
df.dropna(axis = 0, inplace = True)
df.head()

,Unnamed: 0,description,transcription
0,0,23-year-old white female presents complaint al...,"subjective:, 23-year-old white female presents..."
1,1,consult laparoscopic gastric bypass.,"past medical history:, difficulty climbing sta..."
2,2,consult laparoscopic gastric bypass.,"history present illness: , seen abc today. ple..."
3,3,2-d m-mode. doppler.,"2-d m-mode: , ,1. left atrial enlargement left..."
4,4,2-d echocardiogram,1. left ventricular cavity size wall thickness...


In [5]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}

In [6]:
def preprocess(text):
    text = text.lower() # lowercase
    text = text.split() # convert have'nt -> have not
    for i in range(len(text)):
        word = text[i]
        if word in contraction_mapping:
            text[i] = contraction_mapping[word]
    return ' '.join(text)
#define a function for POS tag
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

# define text cleaning process
def clean_text(text):
    ## Remove puncuation
    text=re.sub('[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]',' ',text).lower().split()
    ## Remove stop words
    stop_words = stopwords.words("english")
    text = [w for w in text if w not in stop_words]
    
    # Lemmatization
    wnl=WordNetLemmatizer()
    text=[wnl.lemmatize(i,get_wordnet_pos(i)) for i in text]
    return preprocess(' '.join(text))

sample = "(hello) hi there .man tiger caller who's that isn't it ? WALL-E"
print(preprocess(sample))

(hello) hi there .man tiger caller who is that is not it ? wall-e


In [8]:
#df['transcription'] = df['transcription'].apply(lambda x:preprocess(x))
#df['description'] = df['description'].apply(lambda x:preprocess(x))
#print(df['description'][1],df['transcription'][1])

In [7]:
from sklearn.model_selection import train_test_split
x_tr,x_val,y_tr,y_val=train_test_split(np.array(df['transcription']),np.array(df['description']),test_size=0.1,random_state=0,shuffle=False)

In [8]:
x_tr[2]

'history present illness: , seen abc today. pleasant gentleman 42 years old, 344 pounds. 59". bmi 51. overweight ten years since age 33, highest 358 pounds, lowest 260. pursuing surgical attempts weight loss feel good, get healthy, begin exercise again. wants able exercise play volleyball. physically, sluggish. gets tired quickly. go often. loses weight always regains gains back lost. biggest weight loss 25 pounds three months gained back. six months drinking alcohol taking many calories. multiple commercial weight loss programs including slim fast one month one year ago atkins diet one month two years ago.,past medical history: , difficulty climbing stairs, difficulty airline seats, tying shoes, used public seating, difficulty walking, high cholesterol, high blood pressure. asthma difficulty walking two blocks going eight ten steps. sleep apnea snoring. diabetic, medication. joint pain, knee pain, back pain, foot ankle pain, leg foot swelling. hemorrhoids.,past surgical history: , inc

In [9]:
x = pd.Series(x_tr)
y = pd.Series(y_tr)
print(x[50],y[50],sep='\n')

preoperative diagnosis:, prostate cancer.,postoperative diagnosis: , prostate cancer.,operative procedure: , radical retropubic prostatectomy pelvic lymph node dissection.,anesthesia: ,general epidural,estimated blood loss: , 800 cc.,complications: , none.,indications surgery: , 64-year-old man adenocarcinoma prostate confirmed needle biopsies. elected undergo radical retropubic prostatectomy pelvic lymph node dissection. potential complications include, limited to:,1. infection.,2. bleeding.,3. incontinence.,4. impotence.,5. deep venous thrombosis.,6. recurrence cancer.,procedure detail: , epidural anesthesia administered anesthesiologist holding area. preoperative antibiotic also given preoperative holding area. patient taken operating room general lma anesthesia administered. patient shaved prepped using betadine solution. sterile 16-french foley catheter inserted bladder clear urine drain. midline infraumbilical incision performed. rectus fascia opened sharply. perivesical space re

In [10]:
x_v = pd.Series(x_val)
y_v = pd.Series(y_val)

In [12]:
torch.cuda.is_available() 

True

In [11]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [13]:
def readLangs(text, summary, reverse=False):
    print("Reading lines...")
    
    # Split every line into pairs and normalize
    pairs = [[text[i],summary[i]] for i in range(len(text))]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(summary)
        output_lang = Lang(text)
    else:
        input_lang = Lang(text)
        output_lang = Lang(summary)

    return input_lang, output_lang, pairs

In [15]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    #print("Counted words:")
    #print(input_lang.name, input_lang.n_words)
    #print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

In [16]:
input_lang, output_lang, pairs = prepareData( x, y , False)
print(random.choice(pairs))

Reading lines...
Read 4464 sentence pairs
Counting words...
['history present illness:, 67-year-old gentleman presented emergency room chest pain, cough, hemoptysis, shortness breath, recent 30-pound weight loss. ct scan done chest demonstrated bilateral hilar adenopathy extension subcarinal space well large 6-cm right hilar mass, consistent primary lung carcinoma. also question liver metastases time.,operation performed:, fiberoptic bronchoscopy endobronchial biopsies.,the bronchoscope passed airway noted large, friable tumor blocking bronchus intermedius right. tumor extended carina, involving lingula left upper lobe, appearing malignant. approximately 15 biopsies taken tumor.,attention directed left upper lobe lingula. epinephrine already instilled multiple biopsies taken lingula left upper lobe placed separate container histologic review. approximately eight biopsies taken left upper lobe.', 'fiberoptic bronchoscopy endobronchial biopsies. ct scan done chest demonstrated bilateral 

In [17]:
input_lang_v, output_lang_v, pairs_v = prepareData( x_v, y_v , False)

Reading lines...
Read 496 sentence pairs
Counting words...


#### Encoder - GRU model with 300 units. Embedding size - input vocabulary length.
#### Decoder - GRU with Attention layer, 300 units. Embedding size - output (summary) vocabulary length.

In [13]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [14]:
MAX_LENGTH = 5000

In [15]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [16]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [17]:
def indexesFromSentence(lang, sentence):
    arr = []
    for word in sentence.split(' '):
        try:
            arr.append(lang.word2index[word])
        except:
            arr.append(0)
    return arr
            
def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [18]:
teacher_forcing_ratio = 0.5
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [19]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [20]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [21]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    print("Training....")
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iterator in range(1, n_iters + 1):
        if iterator% 1000 == 0:
            print(iterator,"/",n_iters + 1)
        training_pair = training_pairs[iterator - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iterator % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iterator / n_iters),
                                         iterator, iterator / n_iters * 100, print_loss_avg))

        if iterator % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [22]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [23]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [24]:
def evaluateRandomly_y(encoder, decoder, n=10):
    for pair in pairs_v:
        
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [25]:
device = 'cuda'

hidden_size = 300
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 55000, print_every=5000)

Training....
1000 / 55001
2000 / 55001
3000 / 55001
4000 / 55001
5000 / 55001
48m 33s (- 485m 31s) (5000 9%) 5.3104
6000 / 55001
7000 / 55001
8000 / 55001
9000 / 55001
10000 / 55001
98m 11s (- 441m 52s) (10000 18%) 4.9407
11000 / 55001
12000 / 55001
13000 / 55001
14000 / 55001
15000 / 55001
145m 40s (- 388m 28s) (15000 27%) 4.2547
16000 / 55001
17000 / 55001
18000 / 55001
19000 / 55001
20000 / 55001
197m 7s (- 344m 57s) (20000 36%) 3.6081
21000 / 55001
22000 / 55001
23000 / 55001
24000 / 55001
25000 / 55001
246m 32s (- 295m 51s) (25000 45%) 2.9709
26000 / 55001
27000 / 55001
28000 / 55001
29000 / 55001
30000 / 55001
296m 42s (- 247m 15s) (30000 54%) 2.4623
31000 / 55001
32000 / 55001
33000 / 55001
34000 / 55001
35000 / 55001
346m 48s (- 198m 10s) (35000 63%) 2.0956
36000 / 55001
37000 / 55001
38000 / 55001
39000 / 55001
40000 / 55001
394m 54s (- 148m 5s) (40000 72%) 1.8793
41000 / 55001
42000 / 55001
43000 / 55001
44000 / 55001
45000 / 55001
442m 31s (- 98m 20s) (45000 81%) 1.6980
4600

In [26]:
torch.save(encoder1.state_dict(), './enc3.w')
torch.save(attn_decoder1.state_dict(), './att3.w')

In [18]:
evaluateRandomly_y(encoder1, attn_decoder1)

In [28]:
pairs[:5]

[['subjective:, 23-year-old white female presents complaint allergies. used allergies lived seattle thinks worse here. past, tried claritin, zyrtec. worked short time seemed lose effectiveness. used allegra also. used last summer began using two weeks ago. appear working well. used over-the-counter sprays prescription nasal sprays. asthma doest require daily medication think flaring up.,medications: , medication currently ortho tri-cyclen allegra.,allergies: , known medicine allergies.,objective:,vitals: weight 130 pounds blood pressure 124/78.,heent: throat mildly erythematous without exudate. nasal mucosa erythematous swollen. clear drainage seen. tms clear.,neck: supple without adenopathy.,lungs: clear.,assessment:, allergic rhinitis.,plan:,1. try zyrtec instead allegra again. another option use loratadine. think prescription coverage might cheaper.,2. samples nasonex two sprays nostril given three weeks. prescription written well.',
  '23-year-old white female presents complaint al

In [31]:
from datasets import load_metric
import csv

metric = load_metric("rouge")
data1 = pd.read_csv('./mt_samples_preproc.csv',encoding='iso-8859-1') 

transcriptions_unseen = []
transcriptions_seen = []
actual_summaries_unseen = []
actual_summaries_seen = []
predicted_summaries_unseen = []
predicted_summaries_seen = []

transcriptions_unseen.append('transcription')
transcriptions_seen.append('transcription')
actual_summaries_unseen.append('actual_summaries')
actual_summaries_seen.append('actual_summaries')
predicted_summaries_unseen.append('predicted_summaries')
predicted_summaries_seen.append('predicted_summaries')


for pair in pairs_v:

    output_words, attentions = evaluate(encoder1, attn_decoder1, pair[0])
    output_sentence = ' '.join(output_words)
    transcriptions_unseen.append(pair[0])
    predicted_summaries_unseen.append(output_sentence)
    actual_summaries_unseen.append(pair[1])

for pair in pairs:

    output_words, attentions = evaluate(encoder1, attn_decoder1, pair[0])
    output_sentence = ' '.join(output_words)
    transcriptions_seen.append(pair[0])
    predicted_summaries_seen.append(output_sentence)
    actual_summaries_seen.append(pair[1])

rows_unseen = zip(transcriptions_unseen, actual_summaries_unseen, predicted_summaries_unseen)
rows_seen = zip(transcriptions_seen, actual_summaries_seen, predicted_summaries_seen)

In [32]:
with open('mtsamples_output_seen.csv', "w", encoding="utf-8") as f:
    writer = csv.writer(f)
    for row in rows_seen:
        writer.writerow(row)

with open('mtsamples_output_unseen.csv', "w", encoding="utf-8") as f:
    writer = csv.writer(f)
    for row in rows_unseen:
        writer.writerow(row)

In [33]:
metric.compute(predictions=predicted_summaries_seen, references=actual_summaries_seen)

{'rouge1': AggregateScore(low=Score(precision=0.6664090847883721, recall=0.6982480315486486, fmeasure=0.6625535051826185), mid=Score(precision=0.6734664499049955, recall=0.7067742628310387, fmeasure=0.6699195525617391), high=Score(precision=0.680897087915148, recall=0.7155699617583892, fmeasure=0.6780071549588066)),
 'rouge2': AggregateScore(low=Score(precision=0.5686601254801196, recall=0.6125457714105063, fmeasure=0.5722119367028938), mid=Score(precision=0.5773309542940064, recall=0.6226878163979281, fmeasure=0.5811595889376151), high=Score(precision=0.585384363502145, recall=0.6325548531473143, fmeasure=0.5899670323008036)),
 'rougeL': AggregateScore(low=Score(precision=0.6551274218137426, recall=0.6878311367132569, fmeasure=0.6521948720363728), mid=Score(precision=0.6623554197418038, recall=0.6969978067420624, fmeasure=0.6602393689762978), high=Score(precision=0.6699321471934618, recall=0.7051005104489076, fmeasure=0.6674858661223655)),
 'rougeLsum': AggregateScore(low=Score(precis

In [34]:
metric.compute(predictions=predicted_summaries_unseen, references=actual_summaries_unseen)

{'rouge1': AggregateScore(low=Score(precision=0.4096261106790558, recall=0.4089115837866178, fmeasure=0.38568663368088096), mid=Score(precision=0.434247261667181, recall=0.4368128004601798, fmeasure=0.4106441778098331), high=Score(precision=0.45847381281104793, recall=0.4648930409763885, fmeasure=0.4351587968060159)),
 'rouge2': AggregateScore(low=Score(precision=0.28256497048929574, recall=0.2979496951410391, fmeasure=0.2726358566927594), mid=Score(precision=0.3106235062847339, recall=0.32585437552790775, fmeasure=0.2990527548397035), high=Score(precision=0.33411792985469124, recall=0.3541113590103674, fmeasure=0.32347935456463395)),
 'rougeL': AggregateScore(low=Score(precision=0.39407362954356, recall=0.39507470249746596, fmeasure=0.37108075732826806), mid=Score(precision=0.4201589550411787, recall=0.4237843973307931, fmeasure=0.3975179569043381), high=Score(precision=0.4448608022395026, recall=0.45138295328778544, fmeasure=0.4220999856452086)),
 'rougeLsum': AggregateScore(low=Scor

In [43]:
actual_summaries_unseen[:10]
#transcripts, act, pred = zip(*mapped)
#pred

['actual_summaries',
 'chronic adenotonsillitis adenotonsillar hypertrophy. upper respiratory tract infection mild acute laryngitis.',
 '37-year-old admitted emergency, presented symptoms chest pain, described pressure-type dull ache discomfort precordial region. also, shortness breath noted without diaphoresis. symptoms last 3 4 days especially stress. relation exertional activity. aggravating relieving factors.',
 'patient 63-year-old white male admitted hospital chf lymphedema.',
 'patient multiple problems, main one chest pain night.',
 'patient suffering intractable back leg pain.',
 'patient presents complaint lump upper outer quadrant right breast',
 'patient history right upper pons right cerebral peduncle infarction.',
 'patient 57-year-old female invasive ductal carcinoma left breast, t1c, nx, m0 left breast carcinoma.',
 'patient history mesothelioma likely mild dementia, likely alzheimer type.']

In [44]:
predicted_summaries_unseen[:10]

['predicted_summaries',
 'chronic recurrent upper biopsies. <EOS>',
 'admitted admitted chest male admitted chief ct chest presented symptoms last several days. patients progressive symptoms last night. also noted could noted looked extremely pale. <EOS>',
 'patient 63-year-old male history left chf <EOS>',
 'patient multiple problems, one chest pain night. one week prior admission. <EOS>',
 'patient suffering intractable leg pain. <EOS>',
 'patient presents right arm breast history <EOS>',
 'patient right upper history right infarction. patient states received fresh increased physical right upper extremity pain, patients parents <EOS>',
 'patient 57-year-old female invasive <EOS>',
 'patient history mesothelioma history dementia, <EOS>']